<a href="https://colab.research.google.com/github/Sayed-Husain/Image-Classification-Projects/blob/main/Fresh%20Rotten%20Fruits/fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification: Fresh and Rotten Fruits Classification

In this notebook, we are going to bulid a deep learning computer vision to model to classify fruits into categories: 
1. fresh apples
2. fresh bananas
3. fresh oranges
4. rotten apples
5. rotten bananas
6. rotten oranges

The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)



In [ ]:
!unzip /content/data.zip -d /content

## Load ImageNet Base Model

In [3]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


In [18]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [19]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [20]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

## Augment the Data

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)

## Load Dataset

In [23]:
# load and iterate training dataset
train_it = datagen.flow_from_directory("data/fruits/train/", 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory("data/fruits/valid/", 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1112 images belonging to 6 classes.
Found 323 images belonging to 6 classes.


## Train the Model

In [24]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30
34/34 [==============================] - 23s 635ms/step - loss: 1.4745 - accuracy: 0.5027 - val_loss: 0.7582 - val_accuracy: 0.7554
Epoch 2/30
34/34 [==============================] - 21s 619ms/step - loss: 0.5374 - accuracy: 0.8534 - val_loss: 0.4638 - val_accuracy: 0.8638
Epoch 3/30
34/34 [==============================] - 21s 615ms/step - loss: 0.3471 - accuracy: 0.9074 - val_loss: 0.3457 - val_accuracy: 0.8916
Epoch 4/30
34/34 [==============================] - 21s 609ms/step - loss: 0.2779 - accuracy: 0.9254 - val_loss: 0.2976 - val_accuracy: 0.9040
Epoch 5/30
34/34 [==============================] - 21s 614ms/step - loss: 0.2354 - accuracy: 0.9379 - val_loss: 0.2587 - val_accuracy: 0.9164
Epoch 6/30
34/34 [==============================] - 21s 613ms/step - loss: 0.2134 - accuracy: 0.9460 - val_loss: 0.2390 - val_accuracy: 0.9133
Epoch 7/30
34/34 [==============================] - 21s 612ms/step - loss: 0.1882 - accuracy: 0.9505 - val_loss: 0.2118 - val_accuracy: 0.9319

## Unfreeze Model for Fine Tuning

In [25]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(loss = "categorical_crossentropy",
              optimizer = keras.optimizers.RMSprop(learning_rate = 0.00005),
              metrics = ['accuracy'])

In [28]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=45,
          initial_epoch=30)

Epoch 31/45
34/34 [==============================] - 27s 731ms/step - loss: 0.4206 - accuracy: 0.9649 - val_loss: 0.1324 - val_accuracy: 0.9690
Epoch 32/45
34/34 [==============================] - 25s 712ms/step - loss: 0.0996 - accuracy: 0.9739 - val_loss: 0.0736 - val_accuracy: 0.9752
Epoch 33/45
34/34 [==============================] - 25s 706ms/step - loss: 0.1279 - accuracy: 0.9712 - val_loss: 0.0478 - val_accuracy: 0.9845
Epoch 34/45
34/34 [==============================] - 25s 714ms/step - loss: 0.1165 - accuracy: 0.9694 - val_loss: 0.0605 - val_accuracy: 0.9845
Epoch 35/45
34/34 [==============================] - 25s 710ms/step - loss: 0.0811 - accuracy: 0.9757 - val_loss: 0.0619 - val_accuracy: 0.9876
Epoch 36/45
34/34 [==============================] - 25s 728ms/step - loss: 0.1031 - accuracy: 0.9784 - val_loss: 0.0737 - val_accuracy: 0.9814
Epoch 37/45
34/34 [==============================] - 25s 715ms/step - loss: 0.0845 - accuracy: 0.9883 - val_loss: 0.0273 - val_accuracy:

## Evaluate the Model

In [29]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

10/10 [==============================] - 5s 471ms/step - loss: 0.0443 - accuracy: 0.9845


[0.04426585137844086, 0.9845201373100281]